Domain Basics
=============
In this file, we will explain the usage of the domain classes. 
Every domain is a child of the main class *Domain* and implements the following
methods and properties:

- **__contains__**: to check if a point lays inside this domain
- **volume**, **set_volume**: to compute or set the volume of the domain
- **bounding_box**: to get a bounding box of the given domain
- **boundary**: returns the boundary of the domain, this object itself is a domain and 
                implements the same methods. The boundary also knows the normal 
                vectors. But a boundary itself has **no** boundary!

For explicit documentation of each method and all implemented domains, 
see the docs. Some pre implemented domains are: *Point, Interval, Parallelogram, Circle...*

Here we focus now on the creation of different domains.
To create a domain, we need a space to which the domain belongs. 
The definition of spaces were part of the previous tutorial.

To create the interval [0, 5]:

In [1]:
import torchphysics as tp
T = tp.spaces.R1('t') # we need a one dimensional space
I = tp.domains.Interval(T, 0, 5)

To create the unit square or circle:

In [3]:
X = tp.spaces.R2('x') # we need a two dimensional space
R = tp.domains.Parallelogram(X, [0, 0], [1, 0], [0, 1])
C = tp.domains.Circle(X, [0, 0], 1)
# boundaries:
R_bound = R.boundary
C_bound = C.boundary

Now it would be nice to look at the domains directly. 

In [ ]:
R_sampler = tp.samplers.GridSampler(R, n_points=100) + tp.samplers.GridSampler(R.boundary, n_points=100)
C_sampler = tp.samplers.GridSampler(C, n_points=100) + tp.samplers.GridSampler(C.boundary, n_points=100)

tp.utils.scatter(X, R_sampler)
tp.utils.scatter(X, C_sampler)

The definition of other basic domains, follow the same way.

Domain Operations
-----------------

Until now we only created simple geometries. For complex domains
 the following operations are implemented:

- **Union** $A \cup B$, implemented with: **'+'**
- **Intersection** $A \cap B$, implemented with: **'&'**
- **Cut** $A \setminus B$, implemented with: **'-'**
- **Cartesian product** $A \times B$, implemented with: **'$\cdot$'**

The only important aspect is that for union, intersection and cut the 
domains have to belong to the same space. The returning object is a child of
 *Domains* and implements all previously mentioned methods. 
 The operation can be used for an arbitrary number of times, 
 but for complex domains, the creation of training points will possibly become costly and slow. 

In [7]:
union_domain = R + C
intersection_domain = R & C
cut_domain = R - C

The boundary can be again called with **.boundary**. Since the operation can create 
complex domains the voluem can not always be computed. If a exact value is needed, 
one has to set it over the **set_volume** methode.

Again we can have a look at the corresponding geometries: 

In [ ]:
union_sampler = tp.samplers.GridSampler(union_domain, n_points=100) \
                 + tp.samplers.GridSampler(union_domain.boundary, n_points=100)
inter_sampler = tp.samplers.GridSampler(intersection_domain, n_points=100) \
                 + tp.samplers.GridSampler(intersection_domain.boundary, n_points=100)
cut_sampler = tp.samplers.GridSampler(cut_domain, n_points=142) \
               + tp.samplers.GridSampler(cut_domain.boundary, n_points=100)

tp.utils.scatter(X, union_sampler)
tp.utils.scatter(X, inter_sampler)
tp.utils.scatter(X, cut_sampler)

For the product, we create the Cartesian product of an interval and circle to get a cylinder:

In [ ]:
cylinder = C * I

The product domain will lay in the product of the spaces.
For example, the above cylinder lays now in the space $X \cdot T$

In [ ]:
product_sampler = tp.samplers.GridSampler(C.boundary, n_points=50) * tp.samplers.GridSampler(I, n_points=20)
tp.utils.scatter(X*T, product_sampler)

Changing Domains
----------------
The last aspect, is the possibility of variable domains. E.g. a domain that grows in time or 
a rotating circle. To this end, all parameters that to define a domain can also be function.
For example, the radius of the circle or the origin of a parallelogram.

Right now, a domain can only depend on the variables of another domain. So a domain that should
change with the trained solution is not possible, for now.

The creation of the domains stays the same, one has to only replace the desired parameters.

In [17]:
C = tp.domains.Circle(X, [0, 0], lambda t: t/5)

These domains can be used like all the other domains to create a product domain or apply an operation.

In [ ]:
new_domain = R - C
product_sampler = tp.samplers.GridSampler(new_domain.boundary, density=30) \
                    * tp.samplers.GridSampler(I, n_points=20)
tp.utils.scatter(X*T, product_sampler)